In [ ]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

In [ ]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [ ]:
os.environ["GROQ_API_KEY"] = "gsk_oAoiL......YOUR API KEY"

In [ ]:
# laoding the document
loader = UnstructuredFileLoader("attention_is_all_you_need.pdf")
documents = loader.load()

In [ ]:
# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
type(texts)

list

In [ ]:
len(texts)

26

In [ ]:
#texts[:3]

In [ ]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-10-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
persist_directory = "doc_db"

In [ ]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [ ]:
# retriever
retriever = vectordb.as_retriever()

In [ ]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [ ]:
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# invoke the qa chain and get a response for user query
query = "What is the model architecture discussed in this paper?"
response = qa_chain.invoke({"query": query})

In [ ]:
print(response)

{'query': 'What is the model architecture discussed in this paper?', 'result': 'The model architecture discussed in this paper is the Transformer model. It consists of an encoder and a decoder, both of which are composed of a stack of identical layers. Each layer in the encoder has two sub-layers: a multi-head self-attention mechanism and a position-wise fully connected feed-forward network. The decoder has three sub-layers: a multi-head self-attention mechanism, a multi-head attention mechanism over the output of the encoder stack, and a position-wise fully connected feed-forward network.', 'source_documents': [Document(metadata={'source': 'attention_is_all_you_need.pdf'}, page_content='2\n\nFigure 1: The Transformer - model architecture.\n\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively.\n\n3.1 Encoder and Decoder Stacks

In [ ]:
print(response["result"])

The model architecture discussed in this paper is the Transformer model. It consists of an encoder and a decoder, both of which are composed of a stack of identical layers. Each layer in the encoder has two sub-layers: a multi-head self-attention mechanism and a position-wise fully connected feed-forward network. The decoder has three sub-layers: a multi-head self-attention mechanism, a multi-head attention mechanism over the output of the encoder stack, and a position-wise fully connected feed-forward network.


In [ ]:
print(response["source_documents"][0].metadata["source"])

attention_is_all_you_need.pdf


In [ ]:
# invoke the qa chain and get a response for user query
query = "What is the model architecture discussed in this paper?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

The model architecture discussed in this paper is called the Transformer. It consists of an encoder and a decoder, both of which are composed of a stack of identical layers. 

The encoder has two sub-layers in each layer: 
1. A multi-head self-attention mechanism
2. A position-wise fully connected feed-forward network

The decoder also has these two sub-layers, but it adds a third sub-layer that performs multi-head attention over the output of the encoder stack.

The Transformer uses residual connections and layer normalization around each sub-layer. The output of each sub-layer is computed as LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer itself.
******************************
Source Document: attention_is_all_you_need.pdf
